In [138]:
import pandas as pd
import csv
import nltk
from collections import OrderedDict

training_path = r"C:\Users\Uly\Desktop\Desktop\UCL\IRDM\data\train.csv"
descriptions_path = r"C:\Users\Uly\Desktop\Desktop\UCL\IRDM\data\product_descriptions.csv"
attributes_path = r"C:\Users\Uly\Desktop\Desktop\UCL\IRDM\data\attributes.csv"

In [139]:
df_train = pd.read_csv(training_path, encoding="ISO-8859-1")
df_description = pd.read_csv(descriptions_path, encoding="ISO-8859-1")
df_attributes = pd.read_csv(attributes_path, encoding="ISO-8859-1")

In [148]:
def common_subsequence(doc1, doc2):
    result = []
    d1 = doc1.split(' ')
    d2 = doc2.split(' ')
    
    d2_dict = OrderedDict()
    
    for word in d2:
        d2_dict[word] = word
    
    for word in d1:
        if word in d2_dict:
            result.append(word)
    
    return result

#important nouns in query
def feat_1(product,description,attributes,feat_dict:dict):
    #text = nltk.word_tokenize(product.product_title)
    #tagged_text = nltk.pos_tag(text)
    #print(tagged_text)
    pass
    
#common subsequence between query and title
def feat_2(product,description,attributes,feat_dict:dict):
    query = product.search_term
    title = product.product_title
    subsequence = common_subsequence(query,title)
    result = ' '.join(subsequence)
    feat_dict.update({'subsequence_query+title':result})

#common subsequence between query and description
def feat_3(product,description,attributes,feat_dict:dict):
    query = product.search_term
    desc = description.product_description
    subsequence = common_subsequence(query,desc)
    result = ' '.join(subsequence)
    feat_dict.update({'subsequence_query+description':result})    
    
#all product attributes
def feat_4(product,description,attributes,feat_dict:dict):
    attribute_names = attributes.name
    attribute_values = attributes.value
    
    temp_dict = {}
    for i in range(0, len(attribute_names)):
        temp_dict[attribute_names[i]] = attribute_values[i]
        
    feat_dict.update(temp_dict)
    
#number of words in common between the query and the attributes,and words themselves 
def feat_5_6(product,description,attributes,feat_dict:dict):
    attribute_names = attributes.name
    attribute_values = attributes.value
    
    query = product.search_term
    
    all_words = set()
    
    for i in range(0, len(attribute_names)):
        all_words.add(attribute_names[i])
        all_words.add(attribute_values[i])
        
    common_words = [w if w in all_words for w in query.split(' ')]
    
    if len(common_words) == 0:
        feat_dict.update({'common_word_count':0})
        return
    
    temp_dict = {}
    temp_dict['common_word_count'] = len(common_words)
    for w in common_words:
        temp_dict['common_word:'+w] = True
        
    feat_dict.update(temp_dict)
    
#important nouns in product title
#important nouns in product description

#tf-idf cosine similarity between product title and query
#tf-idf cosine similarity between product description and query

In [149]:
def process_event(product,description,attributes):
    feat_dict = {}
    feat_4(product,description,attributes,feat_dict)
    #...
    return feat_dict
    

In [150]:
def feature_extraction(df_t, df_d, df_a):
    data = []
    for i in range(0, len(df_t)):
        product_row = df_t.iloc[i]
        prod_id = product_row.product_uid
        description_row = df_d.loc[df_d['product_uid'] == prod_id]
        attribute_rows = df_a.loc[df_a['product_uid'] == prod_id]
        data.append(process_event(product_row,description_row,attribute_rows))
        break

In [151]:
feature_extraction(df_train, df_description, df_attributes)

{'Bullet01': 'Versatile connector for various 90Â° connections and home repair projects', 'Bullet02': 'Stronger than angled nailing or screw fastening alone', 'Product Weight (lb.)': '0.26', 'Material': 'Galvanized Steel', 'Product Width (in.)': '3', 'Product Height (in.)': '3', 'Bullet03': 'Help ensure joints are consistently straight and strong', 'Bullet04': 'Dimensions: 3 in. x 3 in. x 1-1/2 in.', 'Product Depth (in.)': '1.5', 'Bullet07': 'Install with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws', 'Bullet06': 'Galvanized for extra corrosion resistance', 'Bullet05': 'Made from 12-Gauge steel', 'Number of Pieces': '1', 'Gauge': '12', 'MFG Brand Name': 'Simpson Strong-Tie'}


In [132]:
common_subsequence('a a a a a','a a a a a a a a a a')

6
11
